# Integration Methods
## 1
### Basic inigrator tester (niet in verslag)
##### used to test if integrator blocks work with the clock as input (x = x)

In [3]:
from CBD.src.pyCBD.lib.std import TimeBlock, ConstantBlock
from pyCBD.simulator import Simulator
from DrawioFiles.generated.Backwards_Euler_Method import *

class basic_intregral_tester(CBD):
    def __init__(self, block_name, block_to_test):
        super().__init__(block_name, input_ports=[], output_ports=['OUT1'])

        # Create the Blocks
        self.addBlock(TimeBlock("Clock_item"))
        self.addBlock(ConstantBlock("Constant_item"))
        self.addBlock(block_to_test)

        # Create the Connections
        self.addConnection("Clock_item", block_to_test.getBlockName(), output_port_name="OUT1", input_port_name="IN1")
        self.addConnection("Constant_item", block_to_test.getBlockName(), output_port_name='OUT1', input_port_name="IC")
        self.addConnection(block_to_test.getBlockName(), "OUT1", output_port_name='OUT1')

### Backwards Euler
##### First generate the python using the draw.io file \
![image info](./DrawioFiles/ImageExports/Backwards_Euler_Method.drawio.png) \
Remake image:

In [4]:
! drawio -x -f png -o ./DrawioFiles/ImageExports/Backwards_Euler_Method.drawio.png ./DrawioFiles/Backwards_Euler_Method.drawio

Checking for beta autoupdate feature for deb/rpm distributions
Found package-type: deb
./DrawioFiles/Backwards_Euler_Method.drawio -> ./DrawioFiles/ImageExports/Backwards_Euler_Method.drawio.png


In [5]:
! python DrawioConvert/__main__.py -F CBD -e backwards_euler_method -sSrgv ./DrawioFiles/Backwards_Euler_Method.drawio -E delta=1 -d ./DrawioFiles/generated/ -f

[  DEBUG  ] Loading Formalism...
[  DEBUG  ] Loaded Formalism.
[  DEBUG  ] Generated './DrawioFiles/generated/Backwards_Euler_Method.py'.
[  DEBUG  ] Generated './DrawioFiles/generated/Backwards_Euler_Method_experiment.py'.
[  DEBUG  ] Done.


In [6]:
from DrawioFiles.generated.Backwards_Euler_Method import *
DELTA_T = 1

backwards_euler_tester = backwards_eurler_method("backwards_euler_tester")
cbd = basic_intregral_tester("tester", backwards_euler_tester)

comperisionBlock = basic_intregral_tester("tester", IntegratorBlock("Intigrator"))

# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(10)

sim = Simulator(comperisionBlock)
sim.setDeltaT(DELTA_T)
sim.run(10)


print(cbd.getSignals())

correct = True
for i, compareItem in enumerate(cbd.getSignals()['OUT1']):
    if compareItem != comperisionBlock.getSignals()['OUT1'][i]:
        correct = False
if correct:
    print("Backward euler is correct")
else:
    print("Backward euler is incorect")
# TODO: Process Your Simulation Results

{'OUT1': [Signal(time=0.0, value=0.0), Signal(time=1.0, value=0.0), Signal(time=2.0, value=1.0), Signal(time=3.0, value=3.0), Signal(time=4.0, value=6.0), Signal(time=5.0, value=10.0), Signal(time=6.0, value=15.0), Signal(time=7.0, value=21.0), Signal(time=8.0, value=28.0), Signal(time=9.0, value=36.0)]}
Backward euler is correct


### Forward Euler
##### First generate the python using the draw.io file
![image info](./DrawioFiles/ImageExports/Forward_Euler_Method.drawio.png) \
remake image:

In [7]:
! drawio -x -f png -o ./DrawioFiles/ImageExports/Forward_Euler_Method.drawio.png ./DrawioFiles/Forward_Euler_Method.drawio

Checking for beta autoupdate feature for deb/rpm distributions
Found package-type: deb
[38589:1106/171425.110269:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 1 times!
./DrawioFiles/Forward_Euler_Method.drawio -> ./DrawioFiles/ImageExports/Forward_Euler_Method.drawio.png


In [8]:
! python DrawioConvert/__main__.py -F CBD -e forward_euler_method -sSrgv ./DrawioFiles/Forward_Euler_Method.drawio -E delta=1 -d ./DrawioFiles/generated/ -f

[  DEBUG  ] Loading Formalism...
[  DEBUG  ] Loaded Formalism.
[  DEBUG  ] Generated './DrawioFiles/generated/Forward_Euler_Method.py'.
[  DEBUG  ] Generated './DrawioFiles/generated/Forward_Euler_Method_experiment.py'.
[  DEBUG  ] Done.


In [9]:
from DrawioFiles.generated.Forward_Euler_Method import *
DELTA_T = 1

forward_euler = forward_euler_method("forward_euler_method")
cbd = basic_intregral_tester("tester", forward_euler)

# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(10)
print(cbd.getSignals())

{'OUT1': [Signal(time=0.0, value=0.0), Signal(time=1.0, value=1.0), Signal(time=2.0, value=3.0), Signal(time=3.0, value=6.0), Signal(time=4.0, value=10.0), Signal(time=5.0, value=15.0), Signal(time=6.0, value=21.0), Signal(time=7.0, value=28.0), Signal(time=8.0, value=36.0), Signal(time=9.0, value=45.0)]}


### Trapiziod rule
##### First generate the python using the draw.io file
![image info](./DrawioFiles/ImageExports/Trapezoid_Rule.drawio.png)
Remake image

In [10]:
! drawio -x -f png -o ./DrawioFiles/ImageExports/Trapezoid_Rule.drawio.png ./DrawioFiles/Trapezoid_Rule.drawio

Checking for beta autoupdate feature for deb/rpm distributions
Found package-type: deb
./DrawioFiles/Trapezoid_Rule.drawio -> ./DrawioFiles/ImageExports/Trapezoid_Rule.drawio.png


In [24]:
! python DrawioConvert/__main__.py -F CBD -e trapezoid_rule -sSrgv ./DrawioFiles/Trapezoid_Rule.drawio -E delta=1 -d ./DrawioFiles/generated/ -f

[  DEBUG  ] Loading Formalism...
[  DEBUG  ] Loaded Formalism.
[  DEBUG  ] Generated './DrawioFiles/generated/Trapezoid_Rule.py'.
[  DEBUG  ] Generated './DrawioFiles/generated/Trapezoid_Rule_experiment.py'.
[  DEBUG  ] Done.


In [28]:
from DrawioFiles.generated.Trapezoid_Rule import *
from pyCBD.simulator import Simulator
        
DELTA_T = 1

trapezoid = trapezoid_rule("trapezoid_rule")
cbd = basic_intregral_tester("trapezoid_euler_method", trapezoid)

# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(10)
print(cbd.getSignals())

{'OUT1': [Signal(time=0.0, value=0.0), Signal(time=1.0, value=0.5), Signal(time=2.0, value=2.0), Signal(time=3.0, value=4.5), Signal(time=4.0, value=8.0), Signal(time=5.0, value=12.5), Signal(time=6.0, value=18.0), Signal(time=7.0, value=24.5), Signal(time=8.0, value=32.0), Signal(time=9.0, value=40.5)]}


## 2 function g(t)
##### First generate the python using the draw.io file
![image info](./DrawioFiles/ImageExports/g_Function.drawio.png)
Remake image:

In [13]:
! drawio -x -f png -o ./DrawioFiles/ImageExports/g_Function.drawio.png ./DrawioFiles/g_Function.drawio

Checking for beta autoupdate feature for deb/rpm distributions
Found package-type: deb
[38818:1106/171433.420480:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 1 times!
./DrawioFiles/g_Function.drawio -> ./DrawioFiles/ImageExports/g_Function.drawio.png


In [14]:
! python DrawioConvert/__main__.py -F CBD -e g_function -sSrgv ./DrawioFiles/g_Function.drawio -E delta=1 -d ./DrawioFiles/generated/ -f

[  DEBUG  ] Loading Formalism...
[  DEBUG  ] Loaded Formalism.
[  DEBUG  ] Generated './DrawioFiles/generated/g_Function.py'.
[  DEBUG  ] Generated './DrawioFiles/generated/g_Function_experiment.py'.
[  DEBUG  ] Done.


#### Create CBD to connect integrators with g(f)

In [15]:
from CBD.src.pyCBD.lib.std import TimeBlock, ConstantBlock
from pyCBD.simulator import Simulator
from DrawioFiles.generated.g_Function import *


class g_function_integrator_tester(CBD):
    def __init__(self, block_name, integrator):
        super().__init__(block_name, input_ports=[], output_ports=['OUT1', 'Gt'])

        # Create the Blocks
        self.addBlock(TimeBlock("Clock_item"))
        self.addBlock(g_function("g_function"))
        self.addBlock(integrator)
        #self.addBlock(g_function("g_function_initial"))
        
        constBlock = ConstantBlock("Constant_item")
        constBlock.setValue(0)
        self.addBlock(constBlock)
        
        # Create the Connections
        self.addConnection("Clock_item", "g_function", output_port_name="OUT1", input_port_name="T")
        #self.addConnection("Constant_item", "g_function_initial", output_port_name='OUT1', input_port_name="T")
        self.addConnection("Constant_item", integrator.getBlockName(), output_port_name='OUT1', input_port_name="IC")
        self.addConnection("g_function", integrator.getBlockName(), output_port_name='OUT1', input_port_name="IN1")
        self.addConnection(integrator.getBlockName(), "OUT1", output_port_name='OUT1')
        self.addConnection("g_function", "Gt", output_port_name='OUT1')



#### Test g(f) on Backwards Euler Method

In [16]:
from DrawioFiles.generated.Backwards_Euler_Method import *
DELTA_T = 0.01

backwards_euler = backwards_eurler_method("backwards_euler")
cbd = g_function_integrator_tester("tester", backwards_euler)


# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(100)


comperisionBlock = g_function_integrator_tester("tester1", IntegratorBlock("Intigrator"))

sim = Simulator(comperisionBlock)
sim.setDeltaT(DELTA_T)
sim.run(100)


correct = True
# 
# for i, compareItem in enumerate(cbd.getSignals()['OUT1']):
#     print(compareItem," != ", comperisionBlock.getSignals()['OUT1'][i])
#     if compareItem != comperisionBlock.getSignals()['OUT1'][i]:
#         correct = False
print(cbd.getSignals()['OUT1'].pop(), comperisionBlock.getSignals()['OUT1'].pop())
if correct:
    print("Backward euler is correct")
else:
    print("Backward euler is incorect")

Signal(time=99.99000000001425, value=3.2134592966575024) Signal(time=99.99000000001425, value=3.2134592966575024)
Backward euler is correct


#### Test g(f) on Forward Euler Method

In [19]:
from DrawioFiles.generated.Backwards_Euler_Method import *
DELTA_T = 0.1

forward_euler = forward_euler_method("forward_euler")
cbd = g_function_integrator_tester("tester", forward_euler)

# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(100)
# 
# for i, compareItem in enumerate(cbd.getSignals()['OUT1']):
#     print(compareItem," != ", comperisionBlock.getSignals()['OUT1'][i])
#     if compareItem != comperisionBlock.getSignals()['OUT1'][i]:
#         correct = False
print(cbd.getSignals()['OUT1'].pop())

Signal(time=99.8999999999986, value=3.2009310596645353)


#### Test g(f) on Trapiziod rule

In [31]:
from DrawioFiles.generated.Backwards_Euler_Method import *
DELTA_T = 1

trapezoid = trapezoid_rule("trapezoid")
cbd = g_function_integrator_tester("tester", trapezoid)

# Run the Simulation
sim = Simulator(cbd)
sim.setDeltaT(DELTA_T)
sim.run(100)

correct = True
# 
# for i, compareItem in enumerate(cbd.getSignals()['OUT1']):
#     print(compareItem," != ", comperisionBlock.getSignals()['OUT1'][i])
#     if compareItem != comperisionBlock.getSignals()['OUT1'][i]:
#         correct = False
print(cbd.getSignals()['OUT1'].pop())
if correct:
    print("Backward euler is correct")
else:
    print("Backward euler is incorect")

Signal(time=99.0, value=3.317050480294049)
Backward euler is correct
